In [ ]:
import pandas as pd

# Load the Excel file to inspect its contents
file_path = "trainingtestingset_latest.xlsx"
xls = pd.ExcelFile(file_path)

# List sheet names to understand the structure
sheet_names = xls.sheet_names

# Load the first few rows of each sheet for inspection
sheet_previews = {sheet: xls.parse(sheet).head() for sheet in sheet_names}
sheet_names, sheet_previews


(['Sheet1'],
 {'Sheet1':    id  taxi_AB  bus_AB  bike_AB  taxi_BC  bus_BC  bike_BC  taxi_CD  bus_CD  \
  0   0       15      15       10        7      13       11       12      20   
  1   1        8      12       15       15      11       12        8      11   
  2   2       12      13       17        8      20       12        7      17   
  3   3        6      17       16       13      10       14       14      18   
  4   4        8      15        9        9      12        8       12      10   
  
     bike_CD  traffic_AB  traffic_BC  traffic_CD  
  0       13           1           2           1  
  1        9           1           2           2  
  2       13           2           2           2  
  3       16           1           2           2  
  4       15           1           1           2  })

In [ ]:
import numpy as np
import itertools
from collections import defaultdict

# Load the sheet
df = xls.parse("Sheet1")

# Define states as tuples of (traffic_AB, traffic_BC, traffic_CD)
df['state'] = list(zip(df['traffic_AB'], df['traffic_BC'], df['traffic_CD']))

# Define all possible actions: each segment (AB, BC, CD) can be taxi, bus, or bike
modes = ['taxi', 'bus', 'bike']
actions = list(itertools.product(modes, repeat=3))  # (mode_AB, mode_BC, mode_CD)

# Reward function: negative of total travel time
def get_travel_time(row, action):
    ab = row[f"{action[0]}_AB"]
    bc = row[f"{action[1]}_BC"]
    cd = row[f"{action[2]}_CD"]
    return ab + bc + cd

# Initialize Q-table
Q = defaultdict(lambda: np.zeros(len(actions)))

# Hyperparameters
alpha = 0.1      # learning rate
gamma = 0.9      # discount factor
epsilon = 1.0    # exploration rate
epsilon_decay = 0.99
num_episodes = 500

# Run Q-learning
for episode in range(num_episodes):
    for i, row in df.iterrows():
        state = row['state']

        if np.random.rand() < epsilon:
            action_idx = np.random.randint(len(actions))
        else:
            action_idx = np.argmax(Q[state])

        action = actions[action_idx]
        reward = -get_travel_time(row, action)  # we want to minimize time

        # No next state transition (static dataset), so max future Q = 0
        Q[state][action_idx] += alpha * (reward - Q[state][action_idx])

    # Decay epsilon
    epsilon *= epsilon_decay

# Extract learned policy
policy = {state: actions[np.argmin(Q[state])] for state in Q}

import pandas as pd
from IPython.display import display

# Convert policy to DataFrame
policy_df = pd.DataFrame([
    {"traffic_AB": s[0], "traffic_BC": s[1], "traffic_CD": s[2],
     "mode_AB": a[0], "mode_BC": a[1], "mode_CD": a[2]}
    for s, a in policy.items()
])

display(policy_df)

,traffic_AB,traffic_BC,traffic_CD,mode_AB,mode_BC,mode_CD
0,1,2,1,bus,bike,bus
1,1,2,2,bike,bike,bus
2,2,2,2,bus,bus,bus
3,1,1,2,bus,bike,bus
4,3,1,3,taxi,bus,bus
5,2,1,2,bike,bike,bus
6,3,3,2,bus,bus,bike
7,1,3,3,taxi,bus,taxi
8,2,2,3,bus,bus,bike
9,1,2,3,bike,bus,bus


# All 100 Rows

In [12]:

# Re-import libraries due to code execution reset
import pandas as pd
import numpy as np
import itertools
from collections import defaultdict

# Reload the Excel file
file_path = "trainingtestingset_latest.xlsx"
xls = pd.ExcelFile(file_path)
df = xls.parse("Sheet1")

# Define state
df['state'] = list(zip(df['traffic_AB'], df['traffic_BC'], df['traffic_CD']))

# Define actions
modes = ['taxi', 'bus', 'bike']
actions = list(itertools.product(modes, repeat=3))

# Reward function
def get_travel_time(row, action):
    ab = row[f"{action[0]}_AB"]
    bc = row[f"{action[1]}_BC"]
    cd = row[f"{action[2]}_CD"]
    return ab + bc + cd

# Q-learning setup
Q = defaultdict(lambda: np.zeros(len(actions)))
alpha = 0.1
gamma = 0.9
epsilon = 1.0
epsilon_decay = 0.99
num_episodes = 500

# Train Q-learning agent
for episode in range(num_episodes):
    for i, row in df.iterrows():
        state = row['state']
        if np.random.rand() < epsilon:
            action_idx = np.random.randint(len(actions))
        else:
            action_idx = np.argmax(Q[state])
        action = actions[action_idx]
        reward = -get_travel_time(row, action)
        Q[state][action_idx] += alpha * (reward - Q[state][action_idx])
    epsilon *= epsilon_decay

# Extract learned policy
policy = {state: actions[np.argmin(Q[state])] for state in Q}

# Apply the learned policy to all 100 rows
df['mode_AB'] = df['state'].apply(lambda s: policy.get(s, ('N/A', 'N/A', 'N/A'))[0])
df['mode_BC'] = df['state'].apply(lambda s: policy.get(s, ('N/A', 'N/A', 'N/A'))[1])
df['mode_CD'] = df['state'].apply(lambda s: policy.get(s, ('N/A', 'N/A', 'N/A'))[2])

from IPython.display import display
display(df[['id', 'traffic_AB', 'traffic_BC', 'traffic_CD', 'mode_AB', 'mode_BC', 'mode_CD']])

df.to_csv("rl_mode_selection_output.csv", index=False)


,id,traffic_AB,traffic_BC,traffic_CD,mode_AB,mode_BC,mode_CD
0,0,1,2,1,bus,bike,bike
1,1,1,2,2,bike,bus,bus
2,2,2,2,2,bus,bus,bus
3,3,1,2,2,bike,bus,bus
4,4,1,1,2,bus,bike,bus
...,...,...,...,...,...,...,...
95,95,1,2,2,bike,bus,bus
96,96,2,2,2,bus,bus,bus
97,97,1,1,2,bus,bike,bus
98,98,1,1,2,bus,bike,bus
